In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


In [ ]:
def fix_and_clean_images(directory):
    num_fixed = 0
    num_removed = 0

    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img = img.convert("RGB")  # Ensure it's RGB
                    img = img.resize((256, 256))  # Resize to standard size
                    img.save(file_path, "JPEG")  # Save as JPG
                    num_fixed += 1
            except (OSError, UnidentifiedImageError):
                print(f"❌ Removing corrupted image: {file_path}")
                os.remove(file_path)
                num_removed += 1

    print(f"✅ Fixed {num_fixed} images and removed {num_removed} corrupted images from {directory}")

In [ ]:
import tensorflow as tf
import os

def resize_images(input_folder, output_folder, size=(224, 224)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for filename in os.listdir(input_folder):
        img_path = os.path.join(input_folder, filename)
        img = tf.io.read_file(img_path)
        img = tf.image.decode_image(img)
        img = tf.image.resize(img, size)
        img = tf.cast(img, tf.uint8)
        tf.io.write_file(os.path.join(output_folder, filename), tf.io.encode_jpeg(img))
    print("Resizing complete!")

resize_images('Tanmay --->  path_to_input_folder', 'Tanmay --->  path_to_output_folder', (224, 224))


In [ ]:
dataset_path = "/kaggle/input/waste-segregation-image-dataset/Dataset"
train_dir = os.path.join(dataset_path, "train")
test_dir = os.path.join(dataset_path, "val")
fix_and_clean_images(train_dir)
fix_and_clean_images(test_dir)
print("Classes:", os.listdir(train_dir))


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)  
BATCH_SIZE = 32  

# Data Augmentation & Normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2 
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

# Check class-to-index mapping
print("Class Indices:", train_generator.class_indices)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
images, labels = next(train_generator)
fig, axes = plt.subplots(3, 3, figsize=(8, 8))
axes = axes.flatten()
for img, ax in zip(images[:9], axes):
    ax.imshow(img)
    ax.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, MaxPooling2D
base_model = ResNet50(weights="/kaggle/input/resnet50-weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5", 
                      include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = MaxPooling2D(pool_size=(2, 2))(x)  
x = GlobalAveragePooling2D()(x) 
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)  
x = MaxPooling2D(pool_size=(2, 2))(x) 
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)  
output_layer = Dense(len(train_generator.class_indices), activation="softmax")(x)  # Output layer

model = Model(inputs=base_model.input, outputs=output_layer)



In [ ]:
for learning_rate in [0.01]:
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                  loss="categorical_crossentropy", 
                  metrics=["accuracy"])
    EPOCHS = 10  

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS,
        verbose=1,
        use_multiprocessing=False  
    )


In [ ]:
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
print("The Plot of Epochs vs accuracy is:")
plt.title("Model Accuracy")
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(learning_rates, accuracies, marker='o')
plt.xscale('log')
plt.xlabel('Learning Rate')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy vs. Learning Rate')
plt.grid(True)
print("The Plot of learning rate vs accuracy is:")
plt.show()

In [ ]:
plt.plot(history.history[""])

In [ ]:
model.save("waste_segregation_model.h5")

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_waste(img_path, model):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0  
    img_array = np.expand_dims(img_array, axis=0)  

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)  
    class_names = list(train_generator.class_indices.keys())  
    return class_names[predicted_class]

test_img = "/kaggle/input/waste-segregation-image-dataset/test/sample.jpg"  # Change this path
predicted_label = predict_waste(test_img, model)
print("Predicted Waste Category:", predicted_label)
